In [30]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk import sent_tokenize
from nltk import word_tokenize
#from rake_nltk import Rake
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import re

In [31]:
df_trainfile=pd.read_csv("D:\\D drive\\Ashish\\Personal\\Python\\Innoplexus\\train_F3WbcTw_1.csv")
df_testfile=pd.read_csv("D:\\D drive\\Ashish\\Personal\\Python\\Innoplexus\\test_tOlRoBf_1.csv")
print(df_trainfile.shape)
print(df_testfile.shape)

(5279, 4)
(2924, 3)


In [32]:
df_trainfile.set_index('unique_hash',inplace=True)
df_testfile.set_index('unique_hash',inplace=True)

In [170]:
df_trainfile["text_drug"]=df_trainfile["text"]



df_trainfile["text_drug"]=df_trainfile["text_drug"].map(lambda x: x.lower())

df_trainfile["text_drug"]=df_trainfile["text_drug"].map(lambda x: re.sub(r'\W', ' ', str(x)))
df_trainfile["text_drug"]=df_trainfile["text_drug"].map(lambda x: re.sub(r'\s+[a-zA-Z]\s+', ' ', x))
df_trainfile["text_drug"]=df_trainfile["text_drug"].map(lambda x: re.sub(r'\^[a-zA-Z]\s+', ' ', x))
df_trainfile["text_drug"]=df_trainfile["text_drug"].map(lambda x: re.sub(r'\s+', ' ', x, flags=re.I))
df_trainfile["text_drug"]=df_trainfile["text_drug"].map(lambda x: re.sub(r'^b\s+', '', x))

In [124]:
print(df_trainfile.sentiment.value_counts())
count_class_2, count_class_1, count_class_0=df_trainfile.sentiment.value_counts()
print((count_class_2))
print((count_class_1))
print((count_class_0))

2    3825
1     837
0     617
Name: sentiment, dtype: int64
3825
837
617


In [125]:
df_class_0 = df_trainfile[df_trainfile['sentiment'] == 0]
df_class_1 = df_trainfile[df_trainfile['sentiment'] == 1]
df_class_2 = df_trainfile[df_trainfile['sentiment'] == 2]

In [126]:
df_class_0_over = df_class_0.sample(count_class_2, replace=True)
df_class_1_over = df_class_1.sample(count_class_2, replace=True)

print(df_class_0_over.shape)
print(df_class_1_over.shape)
print(df_trainfile.shape)

(3825, 4)
(3825, 4)
(5279, 4)


In [127]:
df_test_over = pd.concat([df_class_0_over, df_class_1_over,df_class_2], axis=0)
print(df_test_over.shape)

(11475, 4)


In [128]:
print(df_test_over["sentiment"].value_counts())

lemmer=nltk.stem.WordNetLemmatizer()

def Lem_lemmatize(tokens):
  return [lemmer.lemmatize(token) for token in tokens]

def Lem_tokens(x):
  return Lem_lemmatize(nltk.word_tokenize(x))

clf_tfidf=TfidfVectorizer(lowercase=True,stop_words="english", tokenizer=Lem_tokens)

x_text_train=df_test_over["text_drug"]
x_text_train_trf=clf_tfidf.fit_transform(x_text_train).toarray()

2    3825
1    3825
0    3825
Name: sentiment, dtype: int64


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [129]:
y=df_test_over["sentiment"]

X_train,X_test,y_train,y_test=train_test_split(x_text_train_trf,y,test_size=0.4)

In [122]:
clf_rfc=RandomForestClassifier(random_state=10)

#v_estimators=[50,70,100,150]
v_estimators=[100,150,200]
#v_max_depth=[int(x) for x in np.linspace(start=10,stop=200,num=20)]
#v_max_depth=[50,100,150]
v_max_depth=[100,150,200]
#v_min_samples_leaf=[2,3,4,5]


param_grid={"n_estimators":v_estimators,
            "max_depth":v_max_depth}

clf_gridsearch=GridSearchCV(estimator=clf_rfc,
                           param_grid=param_grid,
                           cv=3,
                           n_jobs=2,
                           verbose=10)

clf_gridsearch.fit(X_train,y_train)
print(clf_gridsearch.best_params_)


Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed: 14.5min


KeyboardInterrupt: 

In [140]:
clf_rfc_f=RandomForestClassifier(random_state=10, n_estimators=200,max_depth=100) 
#-- without min_sample leaf 0.961811076903026
# without drug addition 0.9622244594662654
#0.9597993595429312
clf_rfc_f.fit(X_train,y_train)
y_pred=clf_rfc_f.predict(X_test)

In [141]:
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
print(f1_score(y_test,y_pred,average="macro"))
print(classification_report(y_test,y_pred))

0.9597993595429312
              precision    recall  f1-score   support

           0       0.99      0.98      0.99      1526
           1       0.95      0.96      0.95      1550
           2       0.94      0.95      0.94      1514

   micro avg       0.96      0.96      0.96      4590
   macro avg       0.96      0.96      0.96      4590
weighted avg       0.96      0.96      0.96      4590



In [142]:
print(len(y_test))
#print(y_pred)

df_y_test=pd.DataFrame(data=y_test.values,columns=["Actual"])
df_y_test["Predicted"]=y_pred

print(df_y_test["Predicted"].value_counts())
print(df_y_test["Actual"].value_counts())

4590
1    1560
2    1531
0    1499
Name: Predicted, dtype: int64
1    1550
0    1526
2    1514
Name: Actual, dtype: int64


In [133]:
df_testfile["text_drug"]=df_testfile["text"]
#+" "+df_testfile["drug"]
df_testfile["text_drug"]=df_testfile["text_drug"].map(lambda x: x.lower())
df_testfile["text_drug"]=df_testfile["text_drug"].map(lambda x: re.sub(r'\W', ' ', str(x)))
df_testfile["text_drug"]=df_testfile["text_drug"].map(lambda x: re.sub(r'\s+[a-zA-Z]\s+', ' ', x))
df_testfile["text_drug"]=df_testfile["text_drug"].map(lambda x: re.sub(r'\^[a-zA-Z]\s+', ' ', x))
df_testfile["text_drug"]=df_testfile["text_drug"].map(lambda x: re.sub(r'\s+', ' ', x, flags=re.I))
df_testfile["text_drug"]=df_testfile["text_drug"].map(lambda x: re.sub(r'^b\s+', '', x))

x_text_test=df_testfile["text_drug"]
x_text_test_trf=clf_tfidf.transform(x_text_test).toarray()

In [44]:
print(x_text_test_trf.shape)
print(x_text_train_trf.shape)
print(x_text_test.shape)

(2924, 37532)
(11475, 37532)
(2924,)


In [143]:
y_pred_test=clf_rfc_f.predict(x_text_test_trf)

In [144]:
df_testfile["sentiment"]=y_pred_test

In [146]:
df_testfile["unique_hash"]=df_testfile.index

In [147]:
df_testfile[["unique_hash","sentiment"]].to_csv("D:\\D drive\\Ashish\\Personal\\Python\\Innoplexus\\submission_ashish_v3.csv",index=False)